## Preparing necessary libraries

In [33]:
import numpy as np 
import warnings
from qiskit import QuantumCircuit, transpile
from qiskit.primitives.sampler import Sampler
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator
warnings.filterwarnings('ignore')

## Preparing Quantum Circuit

In [34]:
qc = QuantumCircuit(3)

In [35]:
qc.h(0) # Superposition (Hadamard Gate)
qc.p(np.pi/2,0) # Quantum Phase (Phase Gate)
qc.cx(0,1) # 0th-qubit-Controlle-NOT gate on 1st (CNOT), 1 does operation, 0 does not 
qc.cx(0,2) # 0th-qubit-Controlle-NOT gate on 1st

## Drawing the circuit

In [36]:
qc.draw()

┌───┐┌────────┐          
q_0: ┤ H ├┤ P(π/2) ├──■────■──
     └───┘└────────┘┌─┴─┐  │  
q_1: ───────────────┤ X ├──┼──
                    └───┘┌─┴─┐
q_2: ────────────────────┤ X ├
                         └───┘

## Measurement of all

In [37]:
qc_measured = qc.measure_all(inplace=False)
print(qc_measured)

        ┌───┐┌────────┐           ░ ┌─┐      
   q_0: ┤ H ├┤ P(π/2) ├──■────■───░─┤M├──────
        └───┘└────────┘┌─┴─┐  │   ░ └╥┘┌─┐   
   q_1: ───────────────┤ X ├──┼───░──╫─┤M├───
                       └───┘┌─┴─┐ ░  ║ └╥┘┌─┐
   q_2: ────────────────────┤ X ├─░──╫──╫─┤M├
                            └───┘ ░  ║  ║ └╥┘
meas: 3/═════════════════════════════╩══╩══╩═
                                     0  1  2 


## Sampler

In [38]:
sampler = Sampler()
job = sampler.run(qc_measured, shots=1000000)
result = job.result()
print(f" > Quasi probability distribution: {result.quasi_dists}") # 000 50 % / 111 50 %

 > Quasi probability distribution: [{0: 0.500339, 7: 0.499661}]


## Defining the observable

In [39]:
operator = SparsePauliOp.from_list([("XXY", 1), ("XYX", 1), ("YXX", 1), ("YYY", -1)])
print(operator)

SparsePauliOp(['XXY', 'XYX', 'YXX', 'YYY'],
              coeffs=[ 1.+0.j,  1.+0.j,  1.+0.j, -1.+0.j])


## Estimator

In [40]:
estimator = Estimator()
job = estimator.run(qc, operator, shots=1000)
result = job.result()
print(f" > Expectation values: {result.values}")

 > Expectation values: [4.]


## Transpile

In [41]:
qc_transpiled = transpile(qc, basis_gates = ['cz', 'sx', 'rz'], coupling_map =[[0, 1], [1, 2]] , optimization_level=3)
print(qc_transpiled)

         ┌─────────┐┌────┐┌───────┐             ┌────┐  ┌─────────┐
q_2 -> 0 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π) ├──────■──────┤ √X ├──┤ Rz(π/2) ├
         ├─────────┤├────┤├───────┴┐     │      └────┘  └─────────┘
q_0 -> 1 ┤ Rz(π/2) ├┤ √X ├┤ Rz(-π) ├─■───■─────────────────────────
         ├─────────┤├────┤├───────┬┘ │ ┌────┐┌─────────┐           
q_1 -> 2 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π) ├──■─┤ √X ├┤ Rz(π/2) ├───────────
         └─────────┘└────┘└───────┘    └────┘└─────────┘           
